In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# 建立 Service 物件，指定 chromedriver.exe 的路徑
# service = Service('./chromedriver.exe')

# 設定 Chrome 瀏覽器的選項
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
options.add_argument("--incognito") # 無痕模式
options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。

# 建立 Chrome 瀏覽器物件
driver = webdriver.Chrome(options=options)
driver.get("https://today.line.me/tw/v2/movie/incinemas/playing")

In [ ]:
# driver.execute_script("window.open('about:blank', '_blank');")
# tables = driver.window_handles
# print(tables)
# driver.switch_to.window(tables[-1])
# driver.get("https://today.line.me/tw/v2/movie/incinemas/playing")

['390DAC54729ED6E403223DF94604A2B0', 'F81B5179FBDDE0A26C99C57F119A6AE8', 'D14690B057E0A638E0FE6A33E91582EE', '08C1ACA5E3B362F7C8D22F6C671B4195', '788481DB36D8F9B386515F1080DA2016']


In [67]:
import time
movie_list = driver.find_elements(By.CSS_SELECTOR, ".detailListItem.movieListing-movie")
print(len(movie_list))
last_movie = movie_list[-1]
while True:
    # 捲動到底
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    movie_list = driver.find_elements(By.CSS_SELECTOR, ".detailListItem.movieListing-movie")
    if last_movie == movie_list[-1]:
        break
    last_movie = movie_list[-1]
    print(len(movie_list))

20
30
40
50
60
70
80
90


In [68]:
all_movie_info = []
for movie in movie_list:
    # print(movie_list[-1].find_element(By.CSS_SELECTOR, ".detailListItem-titles").text)
    movie_nameTW = movie.find_element(By.CSS_SELECTOR, ".detailListItem-titles > h2").text
    movie_nameEn = movie.find_element(By.CSS_SELECTOR, ".detailListItem-titles > h3").text
    movie_grade = movie.find_element(By.CSS_SELECTOR, ".iconInfo.detailListItem-iconInfo").text
    movie_duration = movie.find_element(By.CSS_SELECTOR, ".detailListItem-status > span").text
    movie_link = movie.find_element(By.CSS_SELECTOR, ".detailListItem-bookingButton").get_attribute("href")
    all_movie_info.append({
        "NameTW":movie_nameTW, 
        "NameEn":movie_nameEn, 
        "Grade":movie_grade, 
        "Duration":movie_duration, 
        "Link":movie_link
    })
print(all_movie_info[0])
# print(all_movie_info[1])

{'NameTW': '出神入化3', 'NameEn': 'Now You See Me：Now You Don’t', 'Grade': '6.2', 'Duration': '• 1小時53分 • 2025年11月14日上映', 'Link': 'https://today.line.me/tw/v3/movie/JPMx3ox/1'}


In [53]:
# print(all_movie_info[0]['Link'])
driver.get(all_movie_info[0]['Link'])
time.sleep(2)
# 點擊第3個 li，進入 "網友短評"
# driver.find_element(By.CSS_SELECTOR, "ul li:nth-child(3)").click()

In [69]:
def get_all_comment(url):
    driver.get(url)
    driver.find_element(By.CSS_SELECTOR, "ul li:nth-child(3)").click()
    time.sleep(2)
    stars_dict = {
        "M12 18.344l-5.81 3.609c-.147.091-.34.046-.43-.1-.043-.07-.057-.152-.04-.23l1.469-6.96-5.09-4.736c-.126-.118-.133-.316-.016-.442.052-.056.122-.091.198-.099l6.746-.68 2.684-6.513c.066-.16.249-.235.408-.17.077.032.138.094.17.17l2.684 6.514 6.746.68c.172.017.297.17.28.342-.008.075-.043.146-.099.198l-5.09 4.736 1.47 6.96c.036.169-.072.334-.24.37-.08.017-.161.002-.23-.04L12 18.343z":2,
        "M12.12 2.024c.076.031.137.093.169.17l2.684 6.513 6.746.68c.172.017.297.17.28.342-.008.075-.043.146-.099.198l-5.09 4.736 1.47 6.96c.036.169-.072.334-.24.37-.08.017-.161.002-.23-.04L12 18.343l-5.81 3.61c-.147.091-.34.046-.43-.1-.043-.07-.057-.152-.04-.23l1.469-6.96-5.09-4.736c-.126-.118-.133-.316-.016-.442.052-.056.122-.091.198-.099l6.746-.68 2.684-6.513c.066-.16.249-.235.408-.17zM12 6.463v9.651l3.662 2.275-.925-4.383 3.316-3.086-4.398-.443L12 6.463z":1, 
        "M12.12 2.024c.076.031.137.093.169.17l2.684 6.513 6.746.68c.172.017.297.17.28.342-.008.075-.043.146-.099.198l-5.09 4.736 1.47 6.96c.036.169-.072.334-.24.37-.08.017-.161.002-.23-.04L12 18.343l-5.81 3.61c-.147.091-.34.046-.43-.1-.043-.07-.057-.152-.04-.23l1.469-6.96-5.09-4.736c-.126-.118-.133-.316-.016-.442.052-.056.122-.091.198-.099l6.746-.68 2.684-6.513c.066-.16.249-.235.408-.17zm1.6 8.365L12 6.216l-1.72 4.173-4.549.458 3.43 3.191-.961 4.546 3.8-2.36 3.799 2.36-.96-4.546 3.429-3.191-4.548-.458z":0
    }

    comment_info = []

    comment_list = driver.find_elements(By.CSS_SELECTOR, "div.css-hnvcda")

    # testn = 0
    for comment in comment_list:
        name = comment.find_element(By.CSS_SELECTOR, "div.ratingCommentItemUser-name").text
        # print(name)
        
        star_num = 0
        stars = comment.find_elements(By.CSS_SELECTOR, "div.ratingStar > span > svg > path")
        for star in stars:
            # print(star)
            star_num += stars_dict[star.get_attribute('d')]
            # print(star.get_attribute('d'))
        # print(f"{star_num}/10分, {star_num / 2}星")
        
        comment_time = comment.find_element(By.CSS_SELECTOR, ".ratingCommentItemUser-createdTime.css-squ0i5").text
        # print(comment_time)
        # ratingCommentItemContent
        comment_text = comment.find_element(By.CSS_SELECTOR, ".ratingCommentItemContent").text
        # print(comment_text)
        # print('\n')
        comment_info.append(
            {
                "name":name, 
                "score":star_num, 
                "comment_date":comment_time, 
                "comment_text":comment_text
            }
        )
    return comment_info
        # testn += 1
        # if testn == 4:
        #     break

In [70]:
get_all_comment(all_movie_info[0]['Link'])

[{'name': '菲力',
  'score': 5,
  'comment_date': '11月14日22:45',
  'comment_text': '不能說不好看，只是略感失望，舊騎士在整場戲有點多餘，幾乎沒給他們發揮的空間，讓人懷疑導演跟編劇刻意在賣情懷，殘酷一點的說，感覺像是新騎士踩著舊騎士的肩膀上位@@'},
 {'name': 'Syuan🐟',
  'score': 9,
  'comment_date': '11月15日11:04',
  'comment_text': '有種要新舊交替的感覺，但...我還是喜歡原本的五騎士\n9分給我喜歡的五騎士'},
 {'name': 'kkkww Liu',
  'score': 6,
  'comment_date': '11月15日18:04',
  'comment_text': '要推新人來讓ip走遠不反對，可是跟前二集比劇情松散魔術情結都輸前二集太多，真有第四集不會再進電影院觀看了'},
 {'name': '凱鑫企業/玉龍跆拳道 葉佳欣(John)',
  'score': 5,
  'comment_date': '1天前',
  'comment_text': '普普 小失望'},
 {'name': 'WU',
  'score': 8,
  'comment_date': '2天前',
  'comment_text': '好看 邏輯交代滿清楚的 劇情新穎'},
 {'name': 'Wayne',
  'score': 7,
  'comment_date': '2天前',
  'comment_text': '還可以，劇情算是交代的滿清楚的\n帶出新成員'},
 {'name': '昱凱',
  'score': 4,
  'comment_date': '2天前',
  'comment_text': '真的是很難看的續集\n我也是很欣賞原先的5人組合\n但這些知名的5人只在頭尾組合\n中間讓這些新人擔任主角\n我一度在電影院內睡著\n真的不知道編劇為何要做這樣的安排\n很無言💤'},
 {'name': '雯 ꗯ̤̮ ꗯ꙼̈',
  'score': 5,
  'comment_date': '11月15日01:46',
  'comment_text': '略失望'},